In [5]:
%matplotlib inline
import random
import torch
from torch.utils import data
import matplotlib.pyplot as plt

###  生成数据集

In [2]:
def synthetic_data(w,b,num_example):
    X = torch.normal(0,1,(num_example, len(w)))   #随机化X
    y = torch.matmul(X,w)+ b
    y+= torch.normal(0,0.01,y.shape)
    return  X, y.reshape((-1,1))

In [3]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features,labels = synthetic_data(true_w,true_b,1000)

### 读取数据集
因为上一章data_iter读取的数据集效率不高，使用框架中的API进行读取，将features和labels作为API的传递参数，通过数据迭代器指定batch_size。   
布尔值is_train表示数据迭代器对象在每个迭代过程中打乱数据。   
在使用iter构造的python迭代器中，使用next从迭代器中获取数据

1. 使用data.TensorDataset(*data_arrays)进行包装dataset
2. data.DataLoader(dataset,bs,shuffle)进行小批量封装
3. 使用for循环对数据进行提取

In [7]:
def load_array(data_arrays,batch_size,is_train=True):
    '''构造数据迭代器'''
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

In [8]:
batch_size=12
data_iter=load_array((features,labels),batch_size)

In [30]:
for X,y in data_iter:   #for循环直接可以读取
    print(X,y)
    break

tensor([[-0.0262, -0.9219],
        [-0.7125,  0.4159],
        [ 1.3755,  0.1854],
        [ 0.3919,  1.2422],
        [ 1.0526,  1.2540],
        [ 0.5642,  0.1363],
        [-0.1103, -0.9502],
        [-0.7750, -0.9350],
        [-1.2379,  1.8608],
        [-0.4811,  0.5176],
        [ 0.4328, -0.7979],
        [-0.0296,  1.5372]]) tensor([[ 7.2748],
        [ 1.3778],
        [ 6.3286],
        [ 0.7842],
        [ 2.0437],
        [ 4.8727],
        [ 7.1961],
        [ 5.8209],
        [-4.6115],
        [ 1.4772],
        [ 7.7774],
        [-1.0761]])


In [11]:
next(iter(data_iter))

[tensor([[-1.3335,  0.4054],
         [ 1.8135,  0.1199],
         [ 0.9887,  0.5397],
         [ 0.5031, -1.0713],
         [-1.4604,  1.6355],
         [-0.3787, -2.5073],
         [ 2.2434,  1.6072],
         [-2.0869, -1.2086],
         [ 0.0353,  0.3686],
         [-0.2574, -0.6107],
         [ 0.2317,  0.3856],
         [-0.6787,  0.3282]]), tensor([[ 0.1404],
         [ 7.4039],
         [ 4.3430],
         [ 8.8564],
         [-4.2784],
         [11.9676],
         [ 3.2159],
         [ 4.1446],
         [ 2.9995],
         [ 5.7647],
         [ 3.3451],
         [ 1.7441]])]

### 定义模型
调用torch.nn中的Sequential以及全连接层nn.Linear(in_dim,out_dim)

In [22]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

#### 初始化参数
深度学习框架有预定义方式进行初始化参数，本文指定每个权重参数从均值0，标准差0.01的正态分布中随机采样，偏置为0.    
通过net[0]构造网络第一个图层，使用weight.data和bias.data访问参数。使用normal_和fill_来重写参数

In [23]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [24]:
net[0].weight.data

tensor([[ 0.0075, -0.0003]])

In [25]:
net[0].bias.data

tensor([0.])

### 定义损失
均方差的损失定义为MSELoss类，$L_2$范数

In [26]:
loss = nn.MSELoss()

### 定义优化函数
mini-batch的sgd是一种优化的标准工具，调用optim模块实现。     
指定优化参数net().parameters()以及优化算法所以的超参字典。    
本例中lr=0.03

In [28]:
optimizer = torch.optim.SGD(net.parameters(),lr=0.03)

### 训练

In [41]:
num_epochs=3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    l=loss(net(features),labels)
    print(f'epoch{epoch+1},loss{l:f}')

epoch1,loss0.000105
epoch2,loss0.000105
epoch3,loss0.000105
epoch4,loss0.000105
epoch5,loss0.000106
epoch6,loss0.000105
epoch7,loss0.000105
epoch8,loss0.000104
epoch9,loss0.000104
epoch10,loss0.000104


In [42]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))

w的估计误差： tensor([ 0.0003, -0.0005])


In [43]:
b = net[0].bias.data
print('b的估计误差：', true_b - b)

b的估计误差： tensor([-0.0003])


* 我们可以使用PyTorch的高级API更简洁地实现模型。
* 在PyTorch中，`data`模块提供了数据处理工具，`nn`模块定义了大量的神经网络层和常见损失函数。
* 我们可以通过`_`结尾的方法将参数替换，从而初始化参数。

## 练习

1. 如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率？
1. 查看深度学习框架文档，它们提供了哪些损失函数和初始化方法？用Huber损失代替原损失，即
    $$l(y,y') = \begin{cases}|y-y'| -\frac{\sigma}{2} & \text{ if } |y-y'| > \sigma \\ \frac{1}{2 \sigma} (y-y')^2 & \text{ 其它情况}\end{cases}$$
1. 如何访问线性回归的梯度？